In [68]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import os

import logging
logging.getLogger().setLevel(logging.INFO)

from alphabase.spectral_library.flat import SpecLibFlat
from alphabase.spectral_library.base import SpecLibBase
from alphabase.spectral_library.reader import SWATHLibraryReader
from alphadia.extraction.utils import reannotate_fragments
from alphabase.psm_reader import psm_reader_provider

In [182]:
lib_location = '/Users/georgwallmann/Documents/data/alphadia_benchmarking/libraries/marvin_scp/MSfragger_library_mod_noLossType.tsv'
#insilico_lib_location = '/Users/georgwallmann/Documents/data/libraries/2022_111_brunner_2022_1ng_all_29_human_trypsin_diann_decoy.hdf'
output_location = '/Users/georgwallmann/Documents/data/alphadia_benchmarking/libraries/marvin_scp/MSfragger_library_mod_noLossType_d0_d4_d8_separate_eg.hdf'

In [70]:
# create dense library from diann psm file
target_lib = SWATHLibraryReader()
target_lib.add_modification_mapping(
    {'Oxidation@M':['M(Oxidation)'],
     'Dimethyl@K':['K(Dimethyl)'],
     'Dimethyl@R':['R(Dimethyl)'],
     'Dimethyl@Any N-term':['(Dimethyl)']
    }
)

In [71]:
target_lib

In [72]:
psm = target_lib.import_file(lib_location)
target_lib.precursor_df['decoy'] = 0
target_lib._fragment_mz_df = target_lib._fragment_mz_df[['b_z1', 'b_z2', 'y_z1', 'y_z2']]
target_lib._fragment_intensity_df = target_lib._fragment_intensity_df[['b_z1', 'b_z2', 'y_z1', 'y_z2']]

target_lib._precursor_df['elution_group_idx'] = np.arange(len(target_lib._precursor_df))

In [73]:
target_lib._precursor_df['channel'] = 0

In [74]:
target_lib._precursor_df

,sequence,charge,rt,mobility,mods,mod_sites,nAA,frag_start_idx,frag_stop_idx,rt_norm,precursor_mz,ccs,decoy,elution_group_idx,channel
0,AAAAAAAAAPAAAATAPTTAATTAATAAQ,2,45.136304,1.281114,Dimethyl@Any N-term,0,29,0,28,0.454582,1198.624435,516.073311,0,0,0
1,AAAAAAAAAPAAAATAPTTAATTAATAAQ,3,44.985228,1.146438,Dimethyl@Any N-term,0,29,28,56,0.453087,799.418715,692.730690,0,1,0
2,AAAAAAAAAVSR,2,14.846638,0.871435,Dimethyl@Any N-term,0,12,56,67,0.154705,514.795998,353.723212,0,2,0
3,AAAAAAALQAK,2,18.837251,0.892094,Dimethyl@Any N-term;Dimethyl@K,0;11,11,67,77,0.194213,506.811116,362.184412,0,3,0
4,AAAAADLANR,2,13.345655,0.837260,Dimethyl@Any N-term,0,10,77,86,0.139845,486.267073,340.115314,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45863,YYVTIIDAPGHR,2,47.817722,1.075633,Dimethyl@Any N-term,0,12,554086,554097,0.481129,716.882801,434.976907,0,45863,0
45864,YYVTIIDAPGHR,3,47.849555,0.803568,Dimethyl@Any N-term,0,12,554097,554108,0.481444,478.257626,487.431000,0,45864,0
45865,YYYIPQYK,2,47.328657,0.974226,Dimethyl@Any N-term;Dimethyl@K,0;8,8,554108,554115,0.476287,597.315700,394.723226,0,45865,0
45866,YYYQLNSK,2,28.722624,0.965670,Dimethyl@Any N-term;Dimethyl@K,0;8,8,554115,554122,0.292082,567.795132,391.489383,0,45866,0


In [37]:
target_lib.calc_fragment_mz_df()

In [38]:
target_lib.fragment_mz_df

,b_z1,b_z2,y_z1,y_z2,b_modloss_z1,b_modloss_z2,y_modloss_z1,y_modloss_z2
0,106.107507,53.557392,872.516093,436.761685,0.0,0.0,0.0,0.0
1,163.128971,82.068124,815.494629,408.250953,0.0,0.0,0.0,0.0
2,234.166085,117.586681,744.457516,372.732396,0.0,0.0,0.0,0.0
3,371.224996,186.116136,607.398604,304.202940,0.0,0.0,0.0,0.0
4,484.309060,242.658168,494.314540,247.660908,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
91,419.271278,210.139277,553.284120,277.145698,0.0,0.0,0.0,0.0
92,516.324042,258.665659,456.231356,228.619316,0.0,0.0,0.0,0.0
93,653.382954,327.195115,319.172444,160.089860,0.0,0.0,0.0,0.0
94,740.414982,370.711129,232.140416,116.573846,0.0,0.0,0.0,0.0


In [75]:
decoy_lib = SpecLibBase()
decoy_lib._precursor_df = target_lib.precursor_df.copy()
decoy_lib._fragment_mz_df = target_lib._fragment_mz_df.copy()
decoy_lib._fragment_intensity_df = target_lib._fragment_intensity_df.copy()

#decoy_lib.import_file(brunner_lib)
decoy_lib.decoy = 'diann'
decoy_lib.append_decoy_sequence()
decoy_lib.calc_precursor_mz()
decoy_lib.precursor_df = decoy_lib.precursor_df[decoy_lib.precursor_df['decoy'] == 1]

In [76]:
from peptdeep.pretrained_models import ModelManager
from alphabase.peptide.fragment import get_charged_frag_types

frag_types = get_charged_frag_types(
    ['b','y'], 
    2
)

model_mgr = ModelManager()

model_mgr.nce = 30
model_mgr.instrument = 'timsTOF'

In [77]:
res = model_mgr.predict_all(
    decoy_lib.precursor_df,
    predict_items=['ms2'],
    frag_types = frag_types,
)

2023-03-02 10:39:22> Using multiprocessing with 8 processes ...
2023-03-02 10:39:22> Predicting ms2 ...


100%|██████████| 43/43 [00:12<00:00,  3.32it/s]


In [78]:
decoy_lib._precursor_df = res['precursor_df']
decoy_lib._fragment_mz_df = res['fragment_mz_df']
decoy_lib._fragment_intensity_df = res['fragment_intensity_df']

In [79]:
decoy_lib._precursor_df[['frag_start_idx']] += len(target_lib._fragment_mz_df)
decoy_lib._precursor_df[['frag_stop_idx']] += len(target_lib._fragment_mz_df)

In [166]:
output_lib = SpecLibBase()

output_lib._precursor_df = pd.concat([target_lib._precursor_df, decoy_lib._precursor_df], join='inner')
output_lib._fragment_mz_df = pd.concat([target_lib._fragment_mz_df, decoy_lib._fragment_mz_df], join='inner')
output_lib._fragment_intensity_df = pd.concat([target_lib._fragment_intensity_df, decoy_lib._fragment_intensity_df], join='inner')

In [167]:
output_lib._precursor_df = output_lib._precursor_df.sort_values(by=['elution_group_idx']).reset_index(drop=True)
output_lib.precursor_df['precursor_idx'] = np.arange(len(output_lib.precursor_df))
output_lib.precursor_df['elution_group_idx'] = np.arange(len(output_lib.precursor_df))

In [168]:
output_lib.precursor_df['elution_group_idx'].nunique()

89996

In [169]:
output_lib.precursor_df

,sequence,charge,rt,mobility,mods,mod_sites,nAA,frag_start_idx,frag_stop_idx,rt_norm,precursor_mz,ccs,decoy,elution_group_idx,channel,precursor_idx
0,AAAAAAAAAPAAAATAPTTAATTAATAAQ,2,45.136304,1.281114,Dimethyl@Any N-term,0,29,0,28,0.454582,1198.624435,516.073311,0,0,0,0
1,ALAAAAAAAPAAAATAPTTAATTAATALQ,2,45.136304,1.281114,Dimethyl@Any N-term,0,29,1035490,1035518,0.454582,1240.671385,516.073311,1,1,0,1
2,AAAAAAAAAPAAAATAPTTAATTAATAAQ,3,44.985228,1.146438,Dimethyl@Any N-term,0,29,28,56,0.453087,799.418715,692.730690,0,2,0,2
3,ALAAAAAAAPAAAATAPTTAATTAATALQ,3,44.985228,1.146438,Dimethyl@Any N-term,0,29,1036134,1036162,0.453087,827.450015,692.730690,1,3,0,3
4,AAAAAAAAAVSR,2,14.846638,0.871435,Dimethyl@Any N-term,0,12,56,67,0.154705,514.795998,353.723212,0,4,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89991,YYYIPQYK,2,47.328657,0.974226,Dimethyl@Any N-term;Dimethyl@K,0;8,8,554108,554115,0.476287,597.315700,394.723226,0,89991,0,89991
89992,YYYQLNSK,2,28.722624,0.965670,Dimethyl@Any N-term;Dimethyl@K,0;8,8,554115,554122,0.292082,567.795132,391.489383,0,89992,0,89992
89993,YSYQLNTK,2,28.722624,0.965670,Dimethyl@Any N-term;Dimethyl@K,0;8,8,576357,576364,0.292082,536.787307,391.489383,1,89993,0,89993
89994,YSYVPADFVEYDK,2,74.138572,1.123391,Dimethyl@Any N-term;Dimethyl@K,0;13,13,787194,787206,0.741714,826.398147,453.517543,1,89994,0,89994


In [171]:
from tqdm import tqdm
from alphabase.peptide.precursor import get_mod_seq_formula
from alphabase.constants.isotope import IsotopeDistribution


def calc_isotope_dist(precursor_df, max_isotope = 6, min_intensity = 0.001):
    isotope_dist = IsotopeDistribution()

    col_names = ['i_{}'.format(i) for i in range(max_isotope)]
    precursor_df[col_names] = 0.

    for i in tqdm(range(len(precursor_df))):

        row = precursor_df.iloc[i]
        dist, mono = isotope_dist.calc_formula_distribution(
            get_mod_seq_formula(row['sequence'], row['mods'])
        )
        dist[dist < 0.001] = 0.
        dist = dist / dist.sum()
        precursor_df.loc[i, col_names] = dist[:max_isotope]

In [172]:
calc_isotope_dist(output_lib.precursor_df)



100%|██████████| 89996/89996 [00:33<00:00, 2717.73it/s]


In [173]:
channel_lib_4 = SpecLibBase()
channel_lib_4._precursor_df = output_lib.precursor_df.copy()
channel_lib_4._fragment_mz_df = output_lib._fragment_mz_df.copy()
channel_lib_4._fragment_intensity_df = output_lib._fragment_intensity_df.copy()

channel_lib_4._precursor_df['mods'] = channel_lib_4.precursor_df['mods'].apply(lambda x: x.replace('Dimethyl', 'Dimethyl:2H(4)'))
channel_lib_4._precursor_df['channel'] = 4

In [174]:
channel_lib_8 = SpecLibBase()
channel_lib_8._precursor_df = output_lib.precursor_df.copy()
channel_lib_8._fragment_mz_df = output_lib._fragment_mz_df.copy()
channel_lib_8._fragment_intensity_df = output_lib._fragment_intensity_df.copy()

channel_lib_8._precursor_df['mods'] = channel_lib_8.precursor_df['mods'].apply(lambda x: x.replace('Dimethyl', 'Dimethyl:2H(6)13C(2)'))
channel_lib_8._precursor_df['channel'] = 8

In [175]:
def append_lib(
    lib1,
    lib2
):
    lib1._precursor_df = pd.concat([lib1._precursor_df, lib2._precursor_df], join='inner')
    lib1._fragment_mz_df = pd.concat([lib1._fragment_mz_df, lib2._fragment_mz_df], join='inner')
    lib1._fragment_intensity_df = pd.concat([lib1._fragment_intensity_df, lib2._fragment_intensity_df], join='inner')

    lib1._precursor_df[['frag_start_idx']] += len(lib2._fragment_mz_df)
    lib1._precursor_df[['frag_stop_idx']] += len(lib2._fragment_mz_df)

    lib1._precursor_df = lib1._precursor_df.sort_values(by=['elution_group_idx']).reset_index(drop=True)
    lib1.precursor_df['precursor_idx'] = np.arange(len(lib1.precursor_df))


In [176]:
append_lib(output_lib, channel_lib_4)
append_lib(output_lib, channel_lib_8)

In [177]:
output_lib.precursor_df['rt'] = (output_lib.precursor_df['rt'] - output_lib.precursor_df['rt'].min())/ (output_lib.precursor_df['rt'].max() - output_lib.precursor_df['rt'].min())

In [178]:
output_lib.precursor_df['rt'].min()
#output_lib._precursor_df['elution_group_idx'] = np.arange(len(output_lib._precursor_df))

0.0

In [183]:
output_lib._precursor_df['elution_group_idx'] = np.arange(len(output_lib._precursor_df))

In [184]:
output_lib.save_hdf(output_location)

In [27]:
output_lib._precursor_df.iloc[2:5][['sequence', 'charge', 'decoy','mod_seq_charge_hash']]

,sequence,charge,decoy,mod_seq_charge_hash
2,AAAAAAAAAAAAASVGGTIPGPGPGGGQGPGEGGEGQTAR,3,0,2969858240496417433
3,ALAAAAAAAAAAASVGGTIPGPGPGGGQGPGEGGEGQTLR,3,1,2969858240496417433
4,AAAAAAAAAAAAASVGGTIPGPGPGGGQGPGEGGEGQTAR,4,0,2969858240496417434
